In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
# Increase dataframe display limit
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
# Simple method to view the contents of a parquet file
def view_parquet(filename: str):
    df = pd.read_parquet(filename)
    return df

# Used Cars Data Cleaner

Assuming that the raw data (in CSV) has been downloaded, this notebook takes the following class to clean it. 

We will clean the data using the following steps:
1. Turn all columns numeric.
2. Ensure that the index column has string type.
3. Impute missing values with means.
4. Normalize all columns except index and prediction column.
5. Create three splits: subset (used for indexing), train, test). 

## Set variables

In [4]:
# Set the reproducibility seed
seed = 42

In [5]:
# Group columns into several types
boolean_cols = ['frame_damaged', 'has_accidents', 'is_new']
numeric_cols = ['daysonmarket', 'height', 'horsepower', 'length', 'mileage', 'seller_rating']
categorical_cols = []
special_cols = ['price', 'listing_id']
useful_cols = boolean_cols + numeric_cols + categorical_cols + special_cols

## Turn columns numeric

In [6]:
def clean_numeric(value):
    """
    :param value: A value in a pandas cell. 
    :returns: The floating-point representation of the value. 
    """
    # Strings of pattern '--' should be converted to np.nan
    if isinstance(value, str) and value.strip() == '--':
        return np.nan
    
    # For strings of pattern 'XX.XX in', we search for the 'XX.XX' substring via regex
    if isinstance(value, str):
        match = re.search(r'[-+]?[0-9]*\.?[0-9]+', value)
        if match:
            return float(match.group(0))
    
    # Otherwise, try the default pandas to_numeric method
    return pd.to_numeric(value, errors='coerce')

def clean_and_save_csv(input_file, output_file):
    """
    Clean specific string columns that represent numeric concepts, retain useful columns, and save the cleaned DataFrame to an intermediate file. 
    
    :param input_file: Path to the input file. 
    :param output_file: Path to the output file. 
    """
    # Load the file with all columns as strings
    df = pd.read_csv(input_file, dtype=str)
    
    # Retain useful columns
    df = df[useful_cols]
    df = df.loc[:, ~df.columns.duplicated()]
    
    # Convert specific columns to numeric by cleaning them
    for numeric_col in numeric_cols:
        df[numeric_col] = df[numeric_col].map(lambda s: clean_numeric(s))

    for col in boolean_cols:
        # Convert strings like 'True'/'False' and 1/0 into boolean True/False, and anything else into NaN
        df[col] = df[col].apply(lambda x: 1.0 if x in ['True', 'true', '1', 1] else (0.0 if x in ['False', 'false', '0', 0] else np.nan))
    
    # Save the cleaned DataFrame to the output CSV
    df.to_parquet(output_file, index=False)

In [7]:
clean_and_save_csv('/home/jwc/Data/UsedCars/used_cars_data.csv', '/home/jwc/Data/UsedCars/used_cars_data_temp.parquet')

In [8]:
view_parquet('/home/jwc/Data/UsedCars/used_cars_data_temp.parquet')

,frame_damaged,has_accidents,is_new,daysonmarket,height,horsepower,length,mileage,seller_rating,price,listing_id
0,NaN,NaN,1.0,522.0,66.5,177.0,166.6,7.0,2.800000,23141.0,237132766
1,NaN,NaN,1.0,207.0,68.0,246.0,181.0,8.0,3.000000,46500.0,265946296
2,0.0,0.0,0.0,1233.0,58.1,305.0,180.9,NaN,NaN,46995.0,173473508
3,NaN,NaN,1.0,196.0,73.0,340.0,195.1,11.0,3.000000,67430.0,266911050
4,NaN,NaN,1.0,137.0,68.0,246.0,181.0,7.0,3.000000,48880.0,270957414
...,...,...,...,...,...,...,...,...,...,...,...
3000035,0.0,0.0,0.0,16.0,65.4,170.0,183.1,41897.0,4.272727,17998.0,280498781
3000036,NaN,NaN,1.0,171.0,70.7,310.0,204.3,5.0,4.533333,36490.0,269431681
3000037,0.0,1.0,0.0,91.0,58.2,240.0,191.7,57992.0,4.142857,12990.0,274044548
3000038,0.0,0.0,0.0,11.0,55.7,180.0,183.9,27857.0,4.272727,26998.0,280923167


## Encode index as string

In [9]:
# Encode the index column as string to simplify typing in the Python module
df = pd.read_parquet('/home/jwc/Data/UsedCars/used_cars_data_temp.parquet')
df['listing_id'] = df['listing_id'].astype(str)
df.to_parquet('/home/jwc/Data/UsedCars/used_cars_data_temp.parquet', index=False)

## Gather statistics for imputation

In [10]:
def gather_statistics(input_file):
    """
    Gather statistics from the cleaned CSV file:
    1. For numeric columns, compute mean (excluding NaN values).
    2. For categorical columns, compute mode.
    3. For boolean columns, compute the proportion of True values (excluding NaN/None values).
    
    :param input_file: Path to the cleaned CSV file.
    :returns: A dictionary with statistics for each column.
    """
    # Load the file & initialize dictionary
    df = pd.read_parquet(input_file)
    stats = {}
    
    # Process numeric columns
    for col in numeric_cols:
        if col in df.columns:
            mean_value = df[col].mean()
            stats[col] = {'mean': mean_value}
    
    # Process categorical column
    for col in categorical_cols:
        print(col)
        if col in df.columns:
            mode_value = df[col].mode()[0] if not df[col].mode().compute().empty else None
            stats[col] = {'mode': mode_value}
    
    # Process boolean columns
    for col in boolean_cols:
        if col in df.columns:
            mean_value = df[col].mean()
            stats[col] = {'mean': mean_value}
    
    return stats

In [11]:
stats = gather_statistics('/home/jwc/Data/UsedCars/used_cars_data_temp.parquet')
stats

{'daysonmarket': {'mean': 76.05972920361062},
 'height': {'mean': 65.87192975538173},
 'horsepower': {'mean': 247.9957102248012},
 'length': {'mean': 193.6929810577281},
 'mileage': {'mean': 31146.899743421207},
 'seller_rating': {'mean': 4.2704132058199535},
 'frame_damaged': {'mean': 0.009502079831198421},
 'has_accidents': {'mean': 0.1544089561440025},
 'is_new': {'mean': 0.49033912881161584}}

## Impute missing values

In [12]:
def impute_missing_values(input_file, output_file, stats):
    """
    Impute missing values in the cleaned file using the stats.
    
    :param input_file: Path to the input file.
    :param output_file: Path to the output file after imputation.
    :stats: A dictionary with statistics for each column. 
    """
    # Load the file
    df = pd.read_parquet(input_file)

    # Impute numeric columns
    for col in numeric_cols:
        if col in stats and 'mean' in stats[col]:
            mean_value = stats[col]['mean']
            df[col] = df[col].fillna(mean_value)

    # Impute categorical columns
    for col in categorical_cols:
        if col in stats and 'mode' in stats[col]:
            mode_value = stats[col]['mode']
            df[col] = df[col].fillna(mode_value)

    # Impute boolean columns
    for col in boolean_cols:
        mean_value = stats[col]['mean']
        df[col] = df[col].fillna(mean_value)
    
    # Save the DataFrame with imputed values to output file
    df.to_parquet(output_file, index=False)

In [13]:
impute_missing_values('/home/jwc/Data/UsedCars/used_cars_data_temp.parquet', '/home/jwc/Data/UsedCars/used_cars_data_temp.parquet', stats)

In [14]:
view_parquet('/home/jwc/Data/UsedCars/used_cars_data_temp.parquet')

,frame_damaged,has_accidents,is_new,daysonmarket,height,horsepower,length,mileage,seller_rating,price,listing_id
0,0.009502,0.154409,1.0,522.0,66.5,177.0,166.6,7.000000,2.800000,23141.0,237132766
1,0.009502,0.154409,1.0,207.0,68.0,246.0,181.0,8.000000,3.000000,46500.0,265946296
2,0.000000,0.000000,0.0,1233.0,58.1,305.0,180.9,31146.899743,4.270413,46995.0,173473508
3,0.009502,0.154409,1.0,196.0,73.0,340.0,195.1,11.000000,3.000000,67430.0,266911050
4,0.009502,0.154409,1.0,137.0,68.0,246.0,181.0,7.000000,3.000000,48880.0,270957414
...,...,...,...,...,...,...,...,...,...,...,...
3000035,0.000000,0.000000,0.0,16.0,65.4,170.0,183.1,41897.000000,4.272727,17998.0,280498781
3000036,0.009502,0.154409,1.0,171.0,70.7,310.0,204.3,5.000000,4.533333,36490.0,269431681
3000037,0.000000,1.000000,0.0,91.0,58.2,240.0,191.7,57992.000000,4.142857,12990.0,274044548
3000038,0.000000,0.000000,0.0,11.0,55.7,180.0,183.9,27857.000000,4.272727,26998.0,280923167


## Min-max normalize all X columns

In [15]:
def min_max_normalize(input_file, output_file, special_cols=None):
    """
    Normalizes all columns except those specified in special_cols using min-max normalization. 
    
    :param input_file: Path to the input file.
    :param output_file: Path to the output file.
    :param special_cols: List of columns to exclude from normalization.
    """
    # Load the input file
    df = pd.read_parquet(input_file)

    # Select the columns that will be normalized
    if special_cols is None:
        special_cols = []
    columns_to_normalize = [col for col in df.columns if col not in special_cols]

    # Compute min and max for all columns to be normalized
    col_mins = df[columns_to_normalize].min()
    col_maxs = df[columns_to_normalize].max()

    # Normalize each column using min-max normalization
    for col in columns_to_normalize:
        df[col] = (df[col] - col_mins[col]) / (col_maxs[col] - col_mins[col])
    
    # Save the normalized dataframe
    df.to_parquet(output_file, index=False)

In [16]:
min_max_normalize('/home/jwc/Data/UsedCars/used_cars_data_temp.parquet', '/home/jwc/Data/UsedCars/used_cars_data_temp.parquet', special_cols=special_cols)

In [17]:
view_parquet('/home/jwc/Data/UsedCars/used_cars_data_temp.parquet')

,frame_damaged,has_accidents,is_new,daysonmarket,height,horsepower,length,mileage,seller_rating,price,listing_id
0,0.009502,0.154409,1.0,0.145040,0.310391,0.128964,0.321809,7.000001e-08,0.450000,23141.0,237132766
1,0.009502,0.154409,1.0,0.057516,0.330634,0.201903,0.398404,8.000001e-08,0.500000,46500.0,265946296
2,0.000000,0.000000,0.0,0.342595,0.197031,0.264271,0.397872,3.114690e-04,0.817603,46995.0,173473508
3,0.009502,0.154409,1.0,0.054460,0.398111,0.301268,0.473404,1.100000e-07,0.500000,67430.0,266911050
4,0.009502,0.154409,1.0,0.038066,0.330634,0.201903,0.398404,7.000001e-08,0.500000,48880.0,270957414
...,...,...,...,...,...,...,...,...,...,...,...
3000035,0.000000,0.000000,0.0,0.004446,0.295547,0.121564,0.409574,4.189701e-04,0.818182,17998.0,280498781
3000036,0.009502,0.154409,1.0,0.047513,0.367072,0.269556,0.522340,5.000001e-08,0.883333,36490.0,269431681
3000037,0.000000,1.000000,0.0,0.025285,0.198381,0.195560,0.455319,5.799201e-04,0.785714,12990.0,274044548
3000038,0.000000,0.000000,0.0,0.003056,0.164642,0.132135,0.413830,2.785700e-04,0.818182,26998.0,280923167


## Split into train/test/val

In [18]:
# Get the number of rows
df = pd.read_parquet('/home/jwc/Data/UsedCars/used_cars_data_temp.parquet')
n_rows = df.shape[0]
print(f"Number of rows: {n_rows}")

Number of rows: 3000040


In [19]:
def shuffle_and_split(input_file, output_train, output_test, output_val, n, random_state=42):
    """
    Shuffle a dataset and split it into train, test, and validation sets.
    The validation set will have exactly n rows, and the remaining rows will be split 2:1 into train and test sets.

    Parameters:
    input_file (str): Path to the input file (supports Parquet format).
    output_train (str): Path to the output file for the training set.
    output_test (str): Path to the output file for the test set.
    output_val (str): Path to the output file for the validation set.
    n (int): Number of rows for the validation set.
    random_state (int): Random seed for shuffling.
    """
    # Load the data from the input file
    df = pd.read_parquet(input_file)

    # Shuffle the data
    df_shuffled = df.sample(frac=1.0, random_state=random_state).reset_index(drop=True)

    # Create the validation set
    val = df_shuffled.sample(n=n, random_state=random_state)

    # Create a boolean mask for the remaining data
    remaining_mask = ~df_shuffled.index.isin(val.index)
    remaining = df_shuffled[remaining_mask]

    # Split the remaining rows into train and test (2:1 ratio)
    train_size = int(len(remaining) * 2 / 3)
    train = remaining.iloc[:train_size]
    test = remaining.iloc[train_size:]


    # Save the splits to their respective files
    train.to_parquet(output_train, index=False)
    test.to_parquet(output_test, index=False)
    val.to_parquet(output_val, index=False)

In [28]:
shuffle_and_split(
    '/home/jwc/Data/UsedCars/used_cars_data_temp.parquet', 
    '/home/jwc/Data/UsedCars/used_cars_train.parquet', 
    '/home/jwc/Data/UsedCars/used_cars_test.parquet', 
    '/home/jwc/Data/UsedCars/used_cars_val.parquet', 
    100000,
    random_state=seed)

In [29]:
view_parquet('/home/jwc/Data/UsedCars/used_cars_val.parquet')

,frame_damaged,has_accidents,is_new,daysonmarket,height,horsepower,length,mileage,seller_rating,price,listing_id
0,0.009502,0.154409,1.0,0.013337,0.178138,0.140592,0.461702,1.000000e-08,0.803571,21873.0,277478456
1,0.009502,0.154409,1.0,0.002223,0.348178,0.249471,0.482979,0.000000e+00,0.406250,46025.0,280974837
2,0.009502,0.154409,1.0,0.062795,0.192982,0.258985,0.503723,1.650000e-06,0.660714,25934.0,264289554
3,0.009502,0.154409,1.0,0.021673,0.495277,0.332981,0.705851,5.000001e-07,0.776316,63803.0,274964691
4,0.009502,0.154409,1.0,0.048347,0.301915,0.204012,0.465920,1.200000e-07,0.800000,42885.0,269110825
...,...,...,...,...,...,...,...,...,...,...,...
99995,0.000000,0.000000,0.0,0.003890,0.197031,0.126850,0.455319,1.764500e-04,0.935185,18800.0,280433768
99996,0.009502,0.154409,1.0,0.006113,0.345479,0.233615,0.506915,5.000001e-08,0.548077,31631.0,279702245
99997,0.000000,0.000000,0.0,0.019450,0.194332,0.069767,0.393085,2.750000e-06,1.000000,20898.0,275598258
99998,0.000000,0.000000,0.0,0.015560,0.417004,0.317125,0.520745,2.354300e-04,0.652632,43990.0,276801052


In [30]:
view_parquet('/home/jwc/Data/UsedCars/used_cars_train.parquet')

,frame_damaged,has_accidents,is_new,daysonmarket,height,horsepower,length,mileage,seller_rating,price,listing_id
0,0.009502,0.154409,1.0,0.050847,0.288799,0.087738,0.327128,0.000000e+00,0.524390,14890.0,268204976
1,0.009502,0.154409,1.0,0.036121,0.306343,0.228330,0.403723,0.000000e+00,0.875000,39625.0,271352027
2,0.000000,0.000000,0.0,0.012226,0.207827,0.072939,0.404255,3.604900e-04,0.950000,13998.0,277858632
3,0.009502,0.154409,1.0,0.011392,0.180837,0.099366,0.407447,6.000001e-08,0.611111,18549.0,278067159
4,0.000000,0.000000,0.0,0.013337,0.356275,0.238901,0.519149,3.900100e-04,0.875000,26000.0,277457012
...,...,...,...,...,...,...,...,...,...,...,...
1933355,0.000000,0.000000,0.0,0.006113,0.191633,0.137421,0.459574,7.200401e-04,0.875000,16350.0,279863758
1933356,0.009502,0.154409,1.0,0.005279,0.164642,0.108879,0.407447,4.000000e-08,0.500000,21953.0,279956697
1933357,0.009502,0.154409,1.0,0.023896,0.860999,0.140592,0.894681,6.000001e-08,0.825893,49251.0,274389544
1933358,0.009502,0.154409,1.0,0.001667,0.460189,0.359408,0.674468,2.000000e-08,0.806818,52585.0,281185094


In [31]:
view_parquet('/home/jwc/Data/UsedCars/used_cars_test.parquet')

,frame_damaged,has_accidents,is_new,daysonmarket,height,horsepower,length,mileage,seller_rating,price,listing_id
0,0.009502,0.154409,1.0,0.038344,0.384615,0.120507,0.446277,5.000001e-08,0.621324,26603.0,270984036
1,1.000000,1.000000,0.0,0.008336,0.155196,0.089852,0.369149,6.923201e-04,0.916667,7990.0,278957456
2,0.000000,1.000000,0.0,0.001667,0.306343,0.227273,0.403723,2.265100e-04,0.814655,23988.0,281347465
3,0.000000,0.000000,0.0,0.040845,0.301915,0.204012,0.465920,1.315320e-03,0.851852,7985.0,270547934
4,0.009502,0.154409,1.0,0.025563,0.163293,0.101480,0.412234,0.000000e+00,0.944444,29295.0,273833058
...,...,...,...,...,...,...,...,...,...,...,...
966675,0.009502,0.154409,1.0,0.017505,0.437247,0.317125,0.556383,5.000001e-08,0.777778,66100.0,276153709
966676,0.000000,0.000000,0.0,0.000834,0.406208,0.253700,0.602128,1.691150e-03,0.891304,6991.0,281560361
966677,0.009502,0.154409,1.0,0.085301,0.182186,0.784355,0.486170,4.900001e-07,0.633333,74850.0,257433382
966678,0.000000,0.000000,0.0,0.023062,0.191633,0.137421,0.452128,7.353401e-04,0.817603,13995.0,274612190
